<img width="8%" alt="Instagram.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Instagram.png" style="border-radius: 15%">

# Instagram - Extract details from account
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Instagram+-+Extract+details+from+account:+Error+short+description">Bug report</a>

**Tags:** #instagram #snippet #content

**Author:** [Varsha Kumar](https://www.linkedin.com/in/varsha-kumar-590466305/)

**Last update:** 2024-07-09 (Created: 2024-07-04)

**Description:** This notebook allows users to extract details from an Instagram account.

### How to retrive API key with apify

1. Go to https://apify.com.
2. Click "Sign up for free" and use your google account to sign up.
3. Once your account has been created, navigate to "Settings" on the left panel of the screen.
4. Here you will click on the tab labeled "Integrations" where your personal API token that was automatically generated with sign up will be.
5. Copy that token and use it to extract data!

## Input

### Import libraries

In [1]:
import requests
import pandas as pd
import json

### Setup variables
- `apify_token`: personal token apify creates to access data
- `instagram_profile_url`: link to the instagram profile

In [2]:
apify_token = "apify_api_gXWnLEPiE7wC8ALUwQkJ0QcdbuQzU8xxxxxx"
instagram_profile_url = "https://www.instagram.com/naaslife/"
output_csv1 = f"{instagram_profile_url.split('https://www.instagram.com/')[1].replace('/', '_')}instagram_account.csv"
output_csv2 = f"{instagram_profile_url.split('https://www.instagram.com/')[1].replace('/', '_')}instagram_posts.csv"

## Model

### Scrape instagram data

In [3]:
def get_instagram_data(apify_token, instagram_profile_url):
    # Extract the username from the profile URL
    username = instagram_profile_url.split('/')[-2]
    
    # Define the Apify API URL for the Instagram Profile Scraper
    api_url = "https://api.apify.com/v2/acts/apify~instagram-profile-scraper/run-sync-get-dataset-items"

    # Define the payload with the necessary parameters
    payload = {
        "usernames": [username],  # Pass the username as a list
        "proxyConfig": {
            "useApifyProxy": True
        }
    }

    # Define the headers with the Apify API token
    headers = {
        "Authorization": f"Bearer {apify_token}",
        "Content-Type": "application/json"
    }

    # Make the request to the Apify API
    response = requests.post(api_url, json=payload, headers=headers)

    # Extract the JSON data from the response
    data = response.json()
    
    return data

# Make posts dataframe structure
def get_posts(
    ownerUsername,
    ownerId,
    pid,
    post_type,
    caption,
    hashtags,
    mentions,
    url,
    comments_count,
    likes_count,
    timestamp
):
    return {
        "OWNER_USERNAME": ownerUsername,
        "OWNER_ID": ownerId,
        "ID": pid,
        "POST_TYPE": post_type,
        "CAPTION": caption,
        "HASHTAGS": hashtags,
        "MENTIONS": mentions,
        "URL": url,
        "COMMENTS_COUNT": comments_count,
        "LIKES_COUNT": likes_count,
        "TIMESTAMP": timestamp
    }
    
profile_data = get_instagram_data(apify_token, instagram_profile_url)

## Output

### Account dataframe

In [4]:
data1 = []

data1 = [{
        "ID": profile_data[0]['id'],
        "USERNAME": profile_data[0]['username'],
        "URL": profile_data[0]['url'],
        "BIO": profile_data[0]['biography'],
        "FOLLOWERS": profile_data[0]['followersCount'],
        "FOLLOWING": profile_data[0]['followsCount'],
        "PRIVATE": profile_data[0]['private'],
        "POST_COUNT": profile_data[0]['postsCount']
}]

df1 = pd.DataFrame(data1)
df1

### Save first dataframe to csv

In [5]:
df1.to_csv(output_csv1, index=False)

### Posts dataframe

In [6]:
data2 = []

for post in profile_data[0]['latestPosts']:
    data_post = get_posts(
            post["ownerUsername"],
            post["ownerId"],
            post["id"],
            post["type"],
            post["caption"],
            post["hashtags"],
            post["mentions"],
            post["url"],
            post["commentsCount"],
            post["likesCount"],
            post["timestamp"]
        )
    data2.append(data_post)
        
df2 = pd.DataFrame(data2)
df2

### Save second dataframe to csv

In [7]:
df2.to_csv(output_csv2, index=False)